In [ ]:
from PIL import Image
import requests
import shutil
import re
import os
import json

import datetime
from datetime import datetime
from datetime import timedelta

In [ ]:
jhv_api = 'https://api.helioviewer.org/v2/'

In [ ]:
delta = timedelta(seconds = 12)
start = datetime(2017, 9, 11, 0, 0, 0)
end   = datetime(2017, 9, 12, 0, 0, 0)
current = start;
preDate = ""

while end > current:
    payload = {'date': current.isoformat() + "Z", 'sourceId':'10'}
    r = requests.get(jhv_api+'getClosestImage/', payload)
    curDate = json.loads(r.content)['date']
    
    if curDate != preDate:
        print(curDate)
        preDate = curDate
        r = requests.get(jhv_api+'getJP2Image/', payload, stream=True)
        if r.status_code == 200:
            d = r.headers['content-disposition']
            fnames = re.findall("filename=(.+)", d)
            base = os.path.splitext(fnames[0].strip("'").strip("\""))[0]
            with open(base + ".jp2", 'wb') as out_file:
                shutil.copyfileobj(r.raw, out_file)

            img = Image.open(base + ".jp2")
            img.save(base + ".bmp", "BMP")
            img.save(base + ".jpg", "JPEG")
            del img
        del r
    
    current = current + delta